# GBDT(Gradient Boosting Decision Tree) 简介

## 背景知识

- 决策树

- 决策树构造原理
    - CART生成
    - 回归树：均方误差最小
    - 分类树：基尼系数
    
### 回归树
 
#### 回归树的函数表示
 
 $$f(x) = \sum_{m=1}^M c_mI(x \in R_m)$$
 
 M- 叶子节点数
 
 x 属于 Rm则 I为1， 否则为0
 
 #### 最优特征选取
 
 $$min_{j,s}[min_{c1} \sum_{x_1 \in Ri} (y_i - c_1)^2 + min_{c2} \sum_{x_1 \in R_2}(y_i - c_2)^2]$$
 
 j表示特征，s表示特征取值的临界分隔值
 
 $$R_1 = \{x|x^j <= s\},R_2 = \{x|x^j > s\}$$
 
 c1和c2分别代表R1和R2样本平均的label值
 
 #### 树的构建流程
 
 - 遍历所有特征，特征的最佳划分的应的得分，选取最小的分的特征
 
 - 将数据根据此选取的特征分化成两部分
 
 - 在左右两部分中遍历找到划分特征知道满足条件为止
 
### 分类树
 
#### 函数表示
 
与回归树一致
 
#### 最有特征选取
 
- 基尼系数

## 数学原理和构建方法

### 梯度提升树的数学原理

集成学习，采用boosting的方式

#### 提升树的函数表示

$$f_M(x) = \sum_{m=1}^M T(x;\theta_m)$$

M 表示有M颗决策树，$\theta_m$ 表示决策树的参数

#### 学习方式

$$f_m(x) = f_{m-1}(x) +  T(x;\theta_m)$$

即，每次只学习一棵树的参数

$$\theta_m = arg min_{\theta_m} \sum_{i=1}^N L(y_i, f_{m-1}(x_i) + T(x_i;\theta_m))$$

损失函数可以采用平方误差和的方式

$$ L(y, f+m(x)) = [y - f_{m-1}(x) - T(x;\theta_m)]^2 $$

以上每轮迭代的时候，上一轮以前模型都是已知，这样能保证每次都学习一棵树，每次学习都是拟合残差

### 梯度提升树的构建流程

- 初始化$f_0(x) = 0$

- 对于m=1,2...M，计算残差$r_m$，拟合$r_m$，得到$T_m$

- 更新$f_m = f_{m-1} + T_m$

### 梯度提升树残差数值改变

$$r_m = -[\frac{\partial L(y, f(x_i))}{\partial f(x_i)}]_{f_m(x) = f_{m-1}(x)}$$

即，损失函数的负梯度在当前模型的值

## XGBoost数学原理和构建方法

### XGBoost的函数表示

$$f_M(x) = \sum_{m=1}^M T(x;\theta_m)$$

$$f_m(x) = f_{m-1}(x) +  T(x;\theta_m)$$

$$\theta_m = arg min_{\theta_m} \sum_{i=1}^N L(y_i, f_{m-1}(x_i) + T(x_i;\theta_m)) + \Omega(T_m)$$

可以看到很多地方都是和提升树类似的只是添加了正则化项。

- 在XGBoost中我们对优化目标进行泰勒展开：

$$f(x+ \Delta x) = f(x) + f'(x)\Delta x + \frac{1}{2}f''(x) \Delta x^2$$

$$min_{\theta_m}\sum_{i=1}^N[g_i T_m + 0.5 h_i T_m^2] + \Omega(T_m)$$

其中：
$$g_i = \frac{\partial L(y_i,f_{m-1})}{\partial f_{m-1}}, h_i = \frac{\partial^2 L(y_i,f_{m-1})}{\partial f_{m-1}}$$

- 正则化：

XGBoost中都是回归树

$$f(x) = \sum_{j=1}^Q c_j I(x \in R_j)$$

$$\Omega(T_m) = \partial Q + 0.5\beta \sum_{j=1}^Q c_j^2$$

- 优化目标转化：

原目标

$$min_{\theta_m}\sum_{i=1}^N[g_i T_m + 0.5 h_i T_m^2] + \partial Q + 0.5\beta \sum_{j=1}^Q c_j^2$$

转化：

$$ \sum_{j=1}^Q [(\sum_{i \in R}g_i)c_j + 0.5 (\sum_{i \in R} h_i + \beta)c_j^2] + \partial Q$$

- 目标函数最优解：

$$G_j = \sum_{j \in R}g_j, H_i =\sum_{j \in R} h_j $$

$$ \sum_{j=1}^Q [G_jc_j + 0.5 (H_j + \beta)c_j^2] + \partial Q$$

$$c_j = -\frac{G_j}{H_j+\beta}, obj = -0.5\sum_{i=1}^Q\frac{G_j^2}{H_j+\beta} + \partial Q$$

- 最优化分特征选取

$$c_j = -\frac{G_j}{H_j+\beta}, obj = -0.5\sum_{i=1}^Q\frac{G_j^2}{H_j+\beta} + \partial Q$$

$$Gain = (\frac{G_L^2}{H_L + \beta} + \frac{G_R^2}{H_R + \beta} + \frac{(G_L + G_R)^2}{H_L + H_R + \beta}) - \partial$$

### XGBoost 总体流程

- 初始化$f_0(x) = 0$

- 对于m=1,2...M，应用最优化分构造树

- 更新$f_m = f_{m-1} + learn\_rating * T_m$